# Config

In [35]:
import pandas as pd
import geopandas as gpd
import json
import os
from shapely.geometry import Point, GeometryCollection, MultiPolygon

In [3]:
# Example usage
path = "../data/pilot_static_data.csv"  # Replace with the actual path
kpi_path = "../data/cities_kpi.csv"  # Replace with the actual path
geojson_path = "../data/reallocate_all_pilots.geojson"  # Replace with the actual path
output_general = "../config/pilots.json"
output_cities = "../data/pilots/"
output_kpis = "../config/kpi.json"

# Read data

In [5]:
df = pd.read_csv(path,sep="\t")
df.head(1)

,City,Pilot,Thematic Clasters,Impact Areas,Start Date Q,End Date Q,Start Date,End Date,Duration (Months),Description,...,KPIs,Link to Dashboard,UMI Categories,Sources,Tips for replication,Pictures,lat,lon,name,location
0,Gothenburg,Safe System Approach for Children’s Active Tra...,Safe & Sustainable Schools,"Road Safety, Transformative Governance, Inclus...",Q2 2023,Q4 2025,04/01/2023,31/12/2025,35,In the neighbourhood of Bergum Gunnilse key mo...,...,"1, 2, 9",NaN,Modal Share,NaN,NaN,https://reallocatemobility.eu/fileadmin/user_u...,"57,70887","11,97456",Gothenburg (pilot 1),Bergum Gunnilse


In [ ]:
kpi = pd.read_csv(kpi_path,sep=";")
print(kpi.columns)
kpi = kpi[['KPI', 'Name', 'Category', 'Indicator', 'Unit']]
print(kpi.columns)

kpi.head()

In [33]:
geojson = gpd.read_file(geojson_path)
geojson["name"] = geojson["city"] + "(" + geojson["pilot"].str.lower() + ")"
print(geojson.columns)
print(geojson.shape)
geojson.head(1)

Index(['id', 'pilot', 'city', 'geometry', 'name'], dtype='object')
(13, 5)


,id,pilot,city,geometry,name
0,0,Pilot1,Barcelona,"POLYGON ((2.14895 41.37918, 2.14969 41.37854, ...",Barcelona(pilot1)


# General JSON

In [27]:
def extract_all_pilots(df, output_json_path):
    pilots = []
    for _, row in df.iterrows():
        pilot_id = row['name'].lower().replace(" ", "_").replace("(", "").replace(")", "")
        start_date = row["Start Date"]
        if isinstance(start_date, pd.Timestamp):
            start_date = start_date.strftime("%Y-%m-%d")

        # Handle end_date override (if not in standard format)
        end_date = row["End Date"]
        if isinstance(end_date, pd.Timestamp):
            end_date = end_date.strftime("%Y-%m-%d")

        pilot_data = {
            "id": pilot_id,
            "city": row["City"],
            "title": row["Pilot"],
            "name": row["name"],
            "start_date": start_date,
            "end_date": end_date,
            "image": row["Pictures"],
            "lat": row["lat"],
            "lon": row["lon"],
            "thematic": row["Thematic Clasters"],
            "impact_areas": row["Impact Areas"],
            "kpis": row["KPIs"],
            "description": row["Description"]
        }

        pilots.append(pilot_data)

    # Save to JSON
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(pilots, f, ensure_ascii=False, indent=2)

    print(f"Saved {len(pilots)} pilots to {output_json_path}")


In [28]:
extract_all_pilots(df, output_general)

Saved 15 pilots to ../config/pilots.json


# City Json

In [36]:



def save_pilots_as_individual_jsons(df, geojson, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for _, row in df.iterrows():
        pilot_id = row["name"].lower().replace(" ", "_").replace("(", "").replace(")", "")

        try:
            lat = float(str(row["lat"]).replace(",", "."))
            lon = float(str(row["lon"]).replace(",", "."))
        except ValueError:
            print(f"Invalid lat/lon for pilot {row['name']}, skipping.")
            continue
        # Try to get geometry from geojson
        name = row["name"].replace(" ", "")
        matching = geojson[geojson["name"] == name]
        geometries = matching.geometry.tolist()

        # If no geometries are found or they are all None, fall back to Point
        if not geometries or all(g is None for g in geometries):
            geometry = Point(lon, lat)
        else:
            # Combine geometries if more than one
            valid_geometries = [g for g in geometries if g is not None]
            if len(valid_geometries) == 1:
                geometry = valid_geometries[0]
            else:
                # Union or group into GeometryCollection (based on type)
                if all(g.geom_type == "Polygon" for g in valid_geometries):
                    geometry = MultiPolygon(valid_geometries)
                else:
                    geometry = GeometryCollection(valid_geometries)

        # Parse KPIs
        kpis = [k.strip() for k in str(row["KPIs"]).split(",") if k.strip().isdigit()]

        pilot_json = {
            "id": pilot_id,
            "city": row["City"],
            "title": row["Pilot"],
            "image": row["Pictures"],
            "tabs": {
                "Pilot Info": {
                    "description": row["Description"],
                    "location": row["location"],
                    "lat": lat,
                    "lon": lon,
                    "geometry": geometry.__geo_interface__,  # export geometry as GeoJSON-compatible

                },
                "KPIs": kpis,
                "Dashboard": {
                    "title": "",
                    "description": "",
                    "url": row.get("Link to Dashboard", None)
                }
            }
        }

        filename = f"{pilot_id}.json"
        filepath = os.path.join(output_folder, filename)
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(pilot_json, f, ensure_ascii=False, indent=2)

        print(f"Saved: {filepath}")


In [37]:
save_pilots_as_individual_jsons(df,geojson, output_cities)

Saved: ../data/pilots/gothenburg_pilot_1.json
Saved: ../data/pilots/gothenburg_pilot_2.json
Saved: ../data/pilots/heidelberg_pilot_1.json
Saved: ../data/pilots/heidelberg_pilot_2.json
Saved: ../data/pilots/barcelona_pilot_1.json
Saved: ../data/pilots/barcelona_pilot_2.json
Saved: ../data/pilots/budapest_pilot_1.json
Saved: ../data/pilots/budapest_pilot_2.json
Saved: ../data/pilots/lyon_pilot_1.json
Saved: ../data/pilots/lyon_pilot_2.json
Saved: ../data/pilots/bologna.json
Saved: ../data/pilots/utrecht.json
Saved: ../data/pilots/tampere.json
Saved: ../data/pilots/warsaw.json
Saved: ../data/pilots/zagreb.json


## KPI Jsons

In [11]:
def extract_all_kpis(df, output_json_path):
    kpis = []
    
    for _, row in df.iterrows():
        kpi_data = {
            "id": int(row["KPI"]),
            "name": row["Name"],
            "category": row["Category"],
            "indicator": row["Indicator"] if pd.notna(row["Indicator"]) else "",
            "unit": row["Unit"] if pd.notna(row["Unit"]) else ""
        }
        kpis.append(kpi_data)

    # Save to JSON
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(kpis, f, ensure_ascii=False, indent=2)

    print(f"Saved {len(kpis)} KPIs to {output_json_path}")


In [12]:
extract_all_kpis(kpi,output_kpis)

Saved 16 KPIs to ../config/kpi.json
